In [ ]:
import numpy as np
import pandas as pd

from stigmatizing_word_list import STIGMATIZING_WORDS_COMPLETE

In [ ]:
filtered = pd.read_csv("data/clinical_filtered.csv.gz")

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
from tqdm import tqdm

In [ ]:
sentences = []
words = []
for word in tqdm(STIGMATIZING_WORDS_COMPLETE):
    data_word = filtered[filtered.STIG_WORD == word]
    if len(data_word) == 0:
        continue
        
    subject_ids = data_word.SUBJECT_ID.unique()
    
    if len(subject_ids) > 50:
        subject_ids = np.random.choice(subject_ids, 50, replace=False)
    
    for sid in subject_ids:
        subject = data_word.query(f"SUBJECT_ID == {sid}")
        
        if len(subject) >= 1: 
            subject.sample(1)
        else:
            continue
            
        text = subject.TEXT.values[0]

        doc = nlp(text)
        poss_sents = []
        for sent in doc.sents:
            if word in sent.text and sent.text not in sentences:
                poss_sents.append(sent.text)
        if len(poss_sents) >= 1:
            sentences.extend(np.random.choice(poss_sents, 1))
            words.append(word)

In [ ]:
sent_comp = pd.DataFrame({
    "SENTENCE": sentences,
    "STIG_WORD": words
})

In [ ]:
sent_comp.to_csv("data/stig_word_sents.csv", index=False)